In [135]:
import pandas as pd
from dotenv import load_dotenv
import os
import psycopg2
from psycopg2.extras import execute_values
# from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import requests
import re, html


In [136]:
load_dotenv()
DBUSER = os.getenv('DBUSER')
DBPASS = os.getenv('DBPASS')
DBHOST = os.getenv('DBHOST')
DBPORT = os.getenv('DBPORT')
DBNAME = os.getenv('DBNAME')
PUBTABLE = os.getenv('PUBTABLE')
DB_CONFIG = {
    'dbname': DBNAME,
    'user': DBUSER,
    'password': DBPASS,
    'host': DBHOST,
    'port': DBPORT,
}

# Function to establish connection to PostgreSQL
def connect_db(settings):
    conn = psycopg2.connect(**settings)
    return conn

In [131]:
CROSSREF_API_URL = "https://api.crossref.org/works"

# Mapping of Chinese columns to English
COLUMN_MAPPING = {
    '學校單位': 'affiliationTW',
    '姓名': 'correspondingTW',
    '海研一號': 'OR1',
    '海研二號': 'OR2',
    '海研三號': 'OR3',
    '海研五號': 'OR5',
    '新海研1號': 'NOR1',
    '新海研2號': 'NOR2',
    '新海研3號': 'NOR3',
    '勵進': 'LEGEND',
    '新海研1號貴儀中心': 'MIC1',
    '新海研2號貴儀中心': 'MIC2',
    '新海研3號貴儀中心': 'MIC3',
    '海洋學門資料庫': 'ODB'
}

import pandas as pd
import requests
import psycopg2
from psycopg2.extras import execute_values
import re
import html
import time

# Database configuration
DB_CONFIG = {
    'dbname': 'oceanography',
    'user': 'postgres',
    'password': 'your_password',
    'host': 'localhost',
    'port': '5432'
}

CROSSREF_API_URL = "https://api.crossref.org/works"
BATCH_SIZE = 5  # Insert every 5 records
RATE_LIMIT = 1  # Enforce 1 request per second

# Unicode replacements
UNICODE_REPLACEMENTS = {
    # '\u2010': '-',  # Unicode hyphen
    '\u2018': "'",  # Left single quote
    '\u2019': "'",  # Right single quote
    '\u201C': '"',  # Left double quote
    '\u201D': '"',  # Right double quote
    '\u00f1': 'ñ',  # Latin Small Letter Ñ
    '\u00d1': 'Ñ'   # Latin Capital Letter Ñ
}

def clean_title(text):
    text = re.sub(r'<.*?>|\%', '', text)  # Remove HTML-like tags
    text = re.sub(r'\s+', ' ', text)  # Remove HTML-like tags
    text = re.sub(r'\\u2010|\u2013|\u2014|\s*(-)\s*', '', text)  # Convert Unicode hyphen to normal hyphen
    text = re.sub(r'\\u2018|\u2019|\u201C|\u201D|', '', text)    # Convert Unicode single or double quote
    text = re.sub(r'\\u00f1', 'ñ', text)  # Convert Unicode 'ñ'
    text = re.sub(r'\\u00d1', 'Ñ', text)  # Convert Unicode 'Ñ'
    text = re.sub(r'[^\w\s]|[,\.]|[。,\.\?]$', '', text)  # Remove special characters except spaces
    return text.lower().strip()


def format_title_for_db(title: str) -> str:
    """
    Cleans the title field before inserting into the database.
    - Removes HTML tags like <scp>...</scp>
    - Converts Unicode hyphens and special characters
    - Keeps original case formatting
    """
    if not title:
        return ""
    
    # Remove HTML tags
    title = re.sub(r'<.*?>', '', title)
    title = re.sub(r'\s+', ' ', title)

    # Convert Unicode hyphen to regular hyphen
    title = re.sub(r'\u2010|\u2013|\u2014|\s*(-)\s*', "-", title)
    
    # Convert special characters like La Ni\u00f1a → La Niña
    title = title.replace("\u00f1", "ñ").replace("\u00d1", "Ñ")
    
    # Convert Unicode single and double quotes
    title = title.replace("\u2018", "'").replace("\u2019", "'")  # Left & Right single quotes
    title = title.replace("\u201C", '"').replace("\u201D", '"')  # Left & Right double quotes

    # Decode any escaped HTML entities (e.g., &amp;, &lt;)
    title = html.unescape(title)

    # Escape single quotes for PostgreSQL
    return title.replace("'", "''").strip()

# def extract_title(paper_info):
#     match = re.search(r'\(\d{4}.*?\)[.,]?\s*(.*?\.)', paper_info)
#     return match.group(1).strip() if match else paper_info

def extract_title(paper_info):
    """
    Extracts the title from a paper reference, handling different date formats and title endings.
    The title can end with:
    - Chinese period (。)
    - English period (.)
    - Comma (,) when followed by a capitalized word and not inside parentheses
    """
    paper_info = re.sub(r"[‘’“”']", '', paper_info)
 
    # Case 1: Year inside parentheses (any content after year until closing parenthesis is ignored)
    match = re.search(r'[（\(]\d{4}[^）\)]*[）\)][.,]?\s*(.*?)(?:[。.]|,(?![^(]*\))\s*[A-Z])', paper_info)
    if match:
        return match.group(1).strip()

    # Case 2: Year not inside parentheses
    match = re.search(r'\d{4}[.,]?\s*(.*?)(?:[。.]|,(?![^(]*\))\s*[A-Z])', paper_info)
    if match:
        return match.group(1).strip()

    return paper_info.strip()  # Default case: return full text if nothing matches

def fetch_crossref_info(paper_title):
    params = {'query.title': paper_title, 'rows': 5}
    response = requests.get(CROSSREF_API_URL, params=params)
    if response.status_code == 200:
        items = response.json().get('message', {}).get('items', [])
        for item in items:
            if clean_title(paper_title) == clean_title(item.get('title', [''])[0]):
                return item
        print(f"No exact match found for: {paper_title}")
    else:
        print(f"CrossRef API request failed for: {paper_title}")
    return None

def transform_data(row, crossref_data):
    published_date_parts = crossref_data.get('published-print', {}).get('date-parts', [['Unknown']])[0]
    if 'Unknown' in published_date_parts:
        published_date_parts = crossref_data.get('published-online', {}).get('date-parts', [['Unknown']])[0]
 
    print("published_date is: ", published_date_parts, published_date_parts[0], int(published_date_parts[0]) if published_date_parts[0] != 'Unknown' else None)  
    published_date = "-".join(map(str, published_date_parts)) if 'Unknown' not in published_date_parts else 'Unknown'
    published_year = int(published_date_parts[0]) if published_date_parts[0] != 'Unknown' else None
     
    data = {
        'DOI': crossref_data.get('DOI', ''),
        'title': format_title_for_db(crossref_data.get('title', [''])[0]),
        'firstAuthor': crossref_data.get('author', [{}])[0].get('given', '') + ' ' + crossref_data.get('author', [{}])[0].get('family', ''),
        'authors': ', '.join([f"{a.get('given', '')} {a.get('family', '')}" for a in crossref_data.get('author', [])]),
        'publisher': crossref_data.get('publisher', ''),
        'journal': crossref_data.get('short-container-title', [''])[0],
        'published_year': published_year,
        'published_date': published_date,
        'abstract': re.sub(r'<.*?>|Abstract', '', crossref_data.get('abstract', '')),
        'URL': f"https://doi.org/{crossref_data.get('DOI', '')}"
    }
    
    for zh_col, en_col in COLUMN_MAPPING.items():
        if zh_col in row:
            data[en_col] = bool(row[zh_col])
    
    data['affiliationTW'] = row.get('學校單位', '') if pd.notna(row.get('學校單位')) else ''
    data['correspondingTW'] = row.get('姓名', '') if pd.notna(row.get('姓名')) else ''
    
    return data

def create_table():
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    create_table_query = """
    CREATE TABLE IF NOT EXISTS publications (
        DOI TEXT PRIMARY KEY,
        title TEXT,
        firstAuthor TEXT,
        authors TEXT,
        publisher TEXT,
        journal TEXT,
        published_year INT,
        published_date TEXT,
        abstract TEXT,
        URL TEXT,
        affiliationTW TEXT,
        correspondingTW TEXT,
        OR1 BOOLEAN,
        OR2 BOOLEAN,
        OR3 BOOLEAN,
        OR5 BOOLEAN,
        NOR1 BOOLEAN,
        NOR2 BOOLEAN,
        NOR3 BOOLEAN,
        LEGEND BOOLEAN,
        MIC1 BOOLEAN,
        MIC2 BOOLEAN,
        MIC3 BOOLEAN,
        ODB BOOLEAN
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

def doi_exists(doi):
    """ Check if DOI already exists in the database """
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("SELECT 1 FROM publications WHERE DOI = %s", (doi,))
    exists = cursor.fetchone() is not None
    cursor.close()
    conn.close()
    return exists

def process_csv(file_path):
    create_table()
    df = pd.read_csv(file_path)
    processed_data = []

    for _, row in df.iterrows():
        paper_info = row['論文']
        paper_title = extract_title(paper_info)
        crossref_data = fetch_crossref_info(paper_title)
        
        if crossref_data:
            doi = crossref_data.get('DOI', '')
            if not crossref_data.get('DOI'):
                print(f"Warning: Skipping paper '{paper_title}' due to missing DOI.")
                continue

            if doi_exists(doi):
                print(f"⚠️ Warning: DOI {doi} already exists in the database. Skipping insertion.")
                continue  # Skip inserting duplicate records

            processed_data.append(transform_data(row, crossref_data))
        else:
            print(f"Skipping: No match found for {paper_title}")

    insert_into_postgres(processed_data)

def insert_into_postgres(data):
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    insert_query = """
    INSERT INTO publications (
        DOI, title, firstAuthor, authors, publisher, journal,
        published_year, published_date, abstract, URL, affiliationTW, correspondingTW,
        OR1, OR2, OR3, OR5, NOR1, NOR2, NOR3, LEGEND, MIC1, MIC2, MIC3, ODB
    ) VALUES %s
    """
    
    values = [(
        d['DOI'], d['title'], d['firstAuthor'], d['authors'], d['publisher'], d['journal'],
        d['published_year'], d['published_date'], d['abstract'], d['URL'], d['affiliationTW'], d['correspondingTW'],
        d.get('OR1', False), d.get('OR2', False), d.get('OR3', False), d.get('OR5', False),
        d.get('NOR1', False), d.get('NOR2', False), d.get('NOR3', False), d.get('LEGEND', False),
        d.get('MIC1', False), d.get('MIC2', False), d.get('MIC3', False), d.get('ODB', False)
    ) for d in data]

    execute_values(cursor, insert_query, values)
    conn.commit()
    cursor.close()
    conn.close()

In [137]:
conn = psycopg2.connect(**DB_CONFIG)
conn

<connection object at 0x7d6284115d00; dsn: 'user=bioer password=xxx dbname=oceanpub host=192.168.2.37 port=5432', closed: 0>

In [58]:
df = pd.read_csv('../data/test_papers.csv')
df

,序號,論文,出版年份,學校單位,姓名,海研一號,海研二號,海研三號,海研五號,新海研1號,新海研2號,新海研3號,勵進,新海研1號貴儀中心,新海研2號貴儀中心,新海研3號貴儀中心,海洋學門資料庫
0,1,"Tsai, C.-J., M. Andres*, S. Jan*, V. Mensah, T...",2015,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,1
1,2,"Yousuke Kaifu, Tien-Hsia Kuo, Yoshimi Kubota &...",2020,國立臺灣大學海洋研究所,詹森,0,0,0,0,0,0,0,0,0,0,0,1


In [115]:
# Test cases
test_citations = [
    "Tsai, C.-J., M. Andres*, S. Jan*, V. Mensah, T. B. Sanford, R.-C. Lien, and C. M. Lee (2015, Mar), Eddy-Kuroshio interaction processes revealed by mooring observations off Taiwan and Luzon. Geophysical Research...",
    "陳志遠、陳孟仙*（2020年12月）'台江國家公園海域底棲魚類與環境分析'。國家公園學報，30(2), 42-59。",
    "Wang, Y.-H., Lee, I.-H. (2020) Biogeochemistry and dynamics of particulate organic matter in a shallow water hydrothermal field (Kueishantao Islet, NE Taiwan). Marine Geology 422:106121.",
    "Wang, Y.-H., Lee, I.-H. 2020 Biogeochemistry and dynamics of particulate organic matter in a shallow water hydrothermal field (Kueishantao Islet, NE Taiwan). Marine Geology 422:106121.",
    "Yi-Ting Huang (2018, May). Different impact of nanoflagellate grazing and viral lysis on Synechococcus spp and picoeukaryotic mortality in coastal waters, Estuarine, Coastal and Shelf Science (2018, May)",
    "Liu, J.-T., Wang, Y.-H., Lee, I.-H. (2020) Continental shelf morphology controlled by bottom currents, mud diapirism, and submarine slumping to the east of the Gaoping Canyon, off SW Taiwan. Geo-Mar Lett"
]

for citation in test_citations:
    print(extract_title(citation))

Eddy-Kuroshio interaction processes revealed by mooring observations off Taiwan and Luzon
台江國家公園海域底棲魚類與環境分析
Biogeochemistry and dynamics of particulate organic matter in a shallow water hydrothermal field (Kueishantao Islet, NE Taiwan)
Biogeochemistry and dynamics of particulate organic matter in a shallow water hydrothermal field (Kueishantao Islet, NE Taiwan)
Different impact of nanoflagellate grazing and viral lysis on Synechococcus spp and picoeukaryotic mortality in coastal waters
Continental shelf morphology controlled by bottom currents, mud diapirism, and submarine slumping to the east of the Gaoping Canyon, off SW Taiwan


In [116]:
t1 = "Downstream evolution of the Kuroshio's time‐varying transport and velocity structure"
t2 = "Downstream evolution of the <scp>K<\/scp>uroshio's time\u2010varying transport and velocity structure"
print(clean_title(t2))
print(clean_title(t1) == clean_title(t2))
print(clean_title("Continental shelf morphology controlled by bottom currents, mud diapirism, and submarine slumping to the east of the Gaoping Canyon, off SW Taiwan"))
print(clean_title("Neotectonics of the volcanic 'Kuei-Shan Tao' island, and geodynamic implications (NE Taiwan - SW Okinawa Trough)"))
print(clean_title("Three-dimensional coupling between size-fractionated chlorophyll-a, POC and physical processes in the Taiwan Strait in summer"))
print(clean_title("Eddy\u2010Kuroshio Interactions: Local and Remote Effects"))
print(clean_title("Eddy-Kuroshio interactions: Local and remote effects"))
print(clean_title("Vertical Nitrate Flux Induced by Kelvin\u2013Helmholtz Billows Over a Seamount in the Kuroshio"))
print(clean_title("Multivariate analysis of the spatial species diversity of demersal fish assemblages in relation to habitat characteristics in a subtropical national park, Taiwan"))
print(clean_title("Mesozooplankton size structure in response to environmental conditions in the East China Sea: how much does size spectra theory fit empirical data of a dynamic coastal area?."))
print(clean_title("Predator and prey biodiversity relationship and its consequences on marine ecosystem functioning—interplay between nanoflagellates and bacterioplankton"))

downstream evolution of the kuroshios timevarying transport and velocity structure
True
continental shelf morphology controlled by bottom currents mud diapirism and submarine slumping to the east of the gaoping canyon off sw taiwan
neotectonics of the volcanic kueishan tao island and geodynamic implications ne taiwansw okinawa trough
threedimensional coupling between sizefractionated chlorophylla poc and physical processes in the taiwan strait in summer
eddykuroshio interactions local and remote effects
eddykuroshio interactions local and remote effects
vertical nitrate flux induced by kelvinhelmholtz billows over a seamount in the kuroshio
multivariate analysis of the spatial species diversity of demersal fish assemblages in relation to habitat characteristics in a subtropical national park taiwan
mesozooplankton size structure in response to environmental conditions in the east china sea how much does size spectra theory fit empirical data of a dynamic coastal area
predator and prey 

<>:2: SyntaxWarning: invalid escape sequence '\/'
<>:2: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipykernel_16148/903729487.py:2: SyntaxWarning: invalid escape sequence '\/'
  t2 = "Downstream evolution of the <scp>K<\/scp>uroshio's time\u2010varying transport and velocity structure"


In [117]:
print(format_title_for_db("Downstream evolution of the <scp>K</scp>uroshio's time\u2010varying transport and velocity structure"))
print(format_title_for_db("Exceptionally cold water days in the southern Taiwan Strait: their predictability and relation to La Ni\u00f1a"))
print(format_title_for_db("Neotectonics of the volcanic 'Kuei-Shan Tao' island, and geodynamic implications (NE Taiwan - SW Okinawa Trough)"))
print(format_title_for_db("Vertical Nitrate Flux Induced by Kelvin\u2013Helmholtz Billows Over a Seamount in the Kuroshio"))

Downstream evolution of the Kuroshio''s time-varying transport and velocity structure
Exceptionally cold water days in the southern Taiwan Strait: their predictability and relation to La Niña
Neotectonics of the volcanic ''Kuei-Shan Tao'' island, and geodynamic implications (NE Taiwan-SW Okinawa Trough)
Vertical Nitrate Flux Induced by Kelvin-Helmholtz Billows Over a Seamount in the Kuroshio


In [59]:
if True:
    processed_data = []

    for _, row in df.iterrows():
        paper_title = extract_title(row['論文'])
        crossref_data = fetch_crossref_info(paper_title)
        print(crossref_data)
        if crossref_data:
            processed_data.append(transform_data(row, crossref_data))

{'indexed': {'date-parts': [[2024, 8, 2]], 'date-time': '2024-08-02T08:31:25Z', 'timestamp': 1722587485698}, 'reference-count': 25, 'publisher': 'American Geophysical Union (AGU)', 'issue': '19', 'license': [{'start': {'date-parts': [[2015, 10, 8]], 'date-time': '2015-10-08T00:00:00Z', 'timestamp': 1444262400000}, 'content-version': 'vor', 'delay-in-days': 0, 'URL': 'http://onlinelibrary.wiley.com/termsAndConditions#vor'}], 'funder': [{'DOI': '10.13039/501100000730', 'name': 'Museums Association', 'doi-asserted-by': 'publisher', 'award': ['N00014‐15‐1‐2593'], 'id': [{'id': '10.13039/501100000730', 'id-type': 'DOI', 'asserted-by': 'publisher'}]}, {'DOI': '10.13039/100000006', 'name': 'Office of Naval Research', 'doi-asserted-by': 'publisher', 'award': ['N00014‐10‐1‐0397', 'N00014‐10‐1‐0308', 'N00014‐10‐1‐0468'], 'id': [{'id': '10.13039/100000006', 'id-type': 'DOI', 'asserted-by': 'publisher'}]}], 'content-domain': {'domain': [], 'crossmark-restriction': False}, 'short-container-title': 

In [60]:
processed_data

[{'DOI': '10.1002/2015gl065814',
  'title': 'Eddy‐Kuroshio interaction processes revealed by mooring observations off Taiwan and Luzon',
  'firstAuthor': 'Cheng‐Ju Tsai',
  'authors': 'Cheng‐Ju Tsai, Magdalena Andres, Sen Jan, Vigan Mensah, Thomas B. Sanford, Ren‐Chieh Lien, Craig M. Lee',
  'publisher': 'American Geophysical Union (AGU)',
  'journal': 'Geophysical Research Letters',
  'published_year': 2015,
  'published_date': '2015-10-16',
  'abstract': 'The influence and fate of westward propagating eddies that impinge on the Kuroshio were observed with pressure sensor‐equipped inverted echo sounders (PIESs) deployed east of Taiwan and northeast of Luzon. Zero lag correlations between PIES‐measured acoustic travel times and satellite‐measured sea surface height anomalies (SSHa), which are normally negative, have lower magnitude toward the west, suggesting the eddy‐influence is weakened across the Kuroshio. The observational data reveal that impinging eddies lead to seesaw‐like SSHa

In [63]:
process_csv('../data/test_papers.csv')

⚠️ Warning: DOI 10.1002/2015gl065814 already exists in the database. Skipping insertion.
⚠️ Warning: DOI 10.1038/s41598-020-76831-7 already exists in the database. Skipping insertion.


In [139]:
EXPORT_FILE = "../data/publications_export.csv"

def export_publications():
    """Exports the 'publications' table from PostgreSQL to a CSV file."""
    try:
        # Connect to the database
        conn = psycopg2.connect(**DB_CONFIG)
        query = "SELECT * FROM publications;"
        
        # Read the table into a DataFrame
        df = pd.read_sql(query, conn)

        # Export to CSV
        df.to_csv(EXPORT_FILE, index=False, encoding='utf-8')
        
        print(f"Export successful! File saved as {EXPORT_FILE}")
        conn.close()
    
    except Exception as e:
        print(f"Error exporting table: {e}")

    finally:
        print("Export complete.")


# Run the export function
export_publications()


Export successful! File saved as ../data/publications_export.csv
Export complete.


/tmp/ipykernel_16148/2504856620.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [122]:
def clean_abstract(abstract, doi):
    """Fix leading dots or spaces in abstracts using regex."""
    if not isinstance(abstract, str) or not abstract.strip():
        print(f"Warning: Abstract for DOI {doi} is empty or not a string. Skipping.")
        return abstract  # Return as is if empty or not a string

    # Remove **any number** of leading dots and spaces (including mixed cases)
    abstract = re.sub(r'^[.\s]+', '', abstract)

    return abstract.strip()  # Final clean-up

def update_abstracts():
    """Reads all abstracts, fixes formatting, and updates the database."""
    try:
        # Connect to the database
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        # Fetch all abstracts and DOIs
        cursor.execute("SELECT DOI, abstract FROM publications")
        rows = cursor.fetchall()
        
        updated_count = 0
        for doi, abstract in rows:
            fixed_abstract = clean_abstract(abstract, doi)
            
            # Update only if the abstract changed
            if fixed_abstract != abstract:
                cursor.execute("UPDATE publications SET abstract = %s WHERE DOI = %s", (fixed_abstract, doi))
                updated_count += 1
        
        conn.commit()
        print(f"Updated {updated_count} abstracts in the database.")
    
    except Exception as e:
        print(f"Error updating abstracts: {e}")

    finally:
        cursor.close()
        conn.close()

# Run the update function
update_abstracts()

Updated 21 abstracts in the database.


In [ ]:
# Load original input (papers.csv)
df_papers = pd.read_csv("../data/ver_bak/papers_original.csv")
df_exported = pd.read_csv("../data/publications_export.csv")

# Create a mapping of cleaned titles to original titles
original_titles_map = {clean_title(extract_title(title)): title for title in df_papers["論文"]}

# Extract cleaned titles from both files
papers_titles_cleaned = set(original_titles_map.keys())
exported_titles_cleaned = set(clean_title(title) for title in df_exported["title"])

# Find missing cleaned titles
missing_titles_cleaned = papers_titles_cleaned - exported_titles_cleaned

# Retrieve original titles that correspond to missing cleaned titles
missing_titles_original = [original_titles_map[title] for title in missing_titles_cleaned]

# Save missing original titles for reprocessing
missing_titles_df = pd.DataFrame({"論文": missing_titles_original})
missing_titles_df.to_csv("../data/missing_titles.csv", index=False)

print(f"Found {len(missing_titles_original)} missing titles. Saved to missing_titles.csv")

Found 76 missing titles. Saved to missing_titles.csv


In [132]:
def search_by_doi(doi):
    """
    Queries CrossRef API using DOI and inserts data into the 'publications' table if it's not a duplicate.
    """
    url = f"{CROSSREF_API_URL}/doi/{doi}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Error: DOI '{doi}' not found or CrossRef API issue (Status {response.status_code})")
        return None

    crossref_data = response.json().get('message', {})
    
    # Validate DOI presence
    if not crossref_data.get("DOI"):
        print(f"Warning: DOI '{doi}' has no valid metadata in CrossRef. Skipping insertion.")
        return None

    # Check if DOI already exists in the database
    if doi_exists(doi):
        print(f"Warning: DOI '{doi}' already exists in database. Skipping insertion.")
        return None

    # Transform the data to match our publications schema
    transformed_data = transform_data({}, crossref_data)  # Empty row since no CSV row needed

    # Insert data into database
    insert_into_postgres([transformed_data])  # Insert as a single entry

    print(f"✅ Successfully inserted DOI '{doi}' into database.")

    return transformed_data  # Return data for debugging or confirmation

In [138]:
search_by_doi("10.1029/2022GL101851")

published_date is:  [2023, 4, 28] 2023 2023
✅ Successfully inserted DOI '10.1029/2022GL101851' into database.


{'DOI': '10.1029/2022gl101851',
 'title': 'Strongly Scattering Medium Along Slow Earthquake Fault Zones Based on New Observations of Short-Duration Tremors',
 'firstAuthor': 'A. Toh',
 'authors': 'A. Toh, Y. Capdeville, W.‐C. Chi, S. Ide',
 'publisher': 'American Geophysical Union (AGU)',
 'journal': 'Geophysical Research Letters',
 'published_year': 2023,
 'published_date': '2023-4-28',
 'abstract': 'Tremors are a type of slow earthquake with long‐duration signals compared to ordinary earthquakes. The long signals have been considered to solely reflect their long source process. However, here, we provide evidence suggesting that the source processes of tremors are not always long. We refer to these observations as short‐duration tremors. They were recorded by ocean‐bottom seismometers placed very close to the source. Although these tremors exhibit a short‐duration signal when recorded near the source, they exhibit a typical long‐duration signal elsewhere. Our numerical simulations dem